In [1]:
from gdb2csv import converter
from gdbList import list_layers_with_types
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine,text 
import leafmap

In [3]:
# Define the paths and layer name (comment out either one gdb_path when not in use)
#gdb_path = "data/iB1000.gdb"
gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\iB1000.gdb"
csv_path1 = "data/BUILDINGNAME.csv"
layer_name1 = "BUILDINGNAME"
csv_path2 = "data/BUILDINGRELATEBUILDINGNAME.csv"
layer_name2 = "BUILDINGRELATEBUILDINGNAME"

In [4]:
# Define the db connection parameters
username = "postgres"
password = "12345"
host = "localhost"
database = "iB1000"
port = "5432"

engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{database}")

In [5]:
table_name = "building"
sql = text(f"SELECT * FROM {table_name}")

In [6]:
df = gpd.read_postgis(sql, con=engine.connect(), geom_col="shape") 

In [7]:
df.head()

,objectid,lastupdatedate,buildingid,typeofbuildingblock,baselevel,rooflevel,baseleveldatasource,roofleveldatasource,buildingstatus,certainty,shape_length,shape_area,shape
0,1,2024-05-07 16:00:00+00:00,1910247820,T,8.1,16.8,4.0,4.0,E,1,23.908286,31.450706,"MULTIPOLYGON (((816145.174 831352.931, 816141...."
1,2,2024-05-07 16:00:00+00:00,1910247815,T,8.1,16.1,4.0,4.0,E,1,24.364307,31.712137,"MULTIPOLYGON (((816134.845 831345.483, 816130...."
2,3,2024-05-06 16:00:00+00:00,1910247774,TS,NaN,NaN,NaN,NaN,E,1,16.514416,15.452368,"MULTIPOLYGON (((818016.014 838748.731, 818021...."
3,4,2024-05-02 16:00:00+00:00,1910247696,TS,7.2,10.3,4.0,4.0,E,1,32.868446,53.716489,"MULTIPOLYGON (((831625.175 840983.536, 831624...."
4,5,2024-05-02 16:00:00+00:00,1910247695,TS,7.2,10.4,4.0,4.0,E,1,121.792729,226.482654,"MULTIPOLYGON (((831631.172 840978.743, 831627...."


In [5]:
dbname = "sdb"

try:
    conn = psycopg2.connect(dbname=dbname,
                            user='postgres',
                            password='12345',
                            host='localhost')
    print(f"Connected to {dbname} database")

except:
    print(f"I am unable to connect to the {dbname} database")

Connected to sdb database


In [ ]:
buildingname = gpd.read_postgis(sql, conn, geom_col='shape')

In [ ]:
# List all layers within GDB by calling "list_layers_with_types" function
list_layers_with_types(gdb_path)

In [ ]:
# Convert from GDB to CSV by calling "converter" function
converter(gdb_path, csv_path1, layer_name1)
converter(gdb_path, csv_path2, layer_name2)

In [ ]:
# Read the CSV files into DataFrames
df1 = pd.read_csv(csv_path1)
df2 = pd.read_csv(csv_path2)

# Select only the "BUILDINGNAMEID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df1
df1_subset = df1[["BUILDINGNAMEID",
                  "ENGLISHBUILDINGNAME",
                  "CHINESEBUILDINGNAME"]]

# Merge df2 with the subset of df1 on the "BUILDINGNAMEID" column
df3 = pd.merge(df2, df1_subset, on="BUILDINGNAMEID")

# Display the first few rows of the merged DataFrame
df3.head()

In [ ]:
gdf1 = gpd.read_file(gdb_path, layer="Building")
gdf1.head()

In [ ]:
# Select only the "BUILDINGID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df3
df3_subset = df3[["BUILDINGID",
                  "ENGLISHBUILDINGNAME",
                  "CHINESEBUILDINGNAME"]]

# Merge gdf1 with the subset of df3 on the "BUILDINGID" column
gdf2 = pd.merge(gdf1,
                df3_subset,
                on="BUILDINGID")

# Display the first few rows of the merged DataFrame
gdf2.head()

In [ ]:
#shp_path = "data/building.shp"

#Convert the datetimes to strings
#gdf2['LASTUPDATEDATE'] = gdf2['LASTUPDATEDATE'].dt.strftime('%Y-%m-%d')

#gdf2.to_file(shp_path, encoding='utf-8')

In [ ]:
# Keep only relevant columns
gdf3 = gdf2.loc[:, ("BUILDINGID",
                    "TYPEOFBUILDINGBLOCK",
                    "BASELEVEL",
                    "ROOFLEVEL",
                    "BUILDINGSTATUS",
                    "ENGLISHBUILDINGNAME",
                    "CHINESEBUILDINGNAME")]

gdf3.head(30)